In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.onnx
import onnx
import os

In [2]:
# ==========================================
# 1. 定义 NanoMLP 模型 (核心修改点)
# ==========================================
class NanoMLP(nn.Module):
    def __init__(self):
        super(NanoMLP, self).__init__()
        # 输入层 784 (28x28) -> 隐藏层 32 [cite: 4, 11, 12]
        self.fc1 = nn.Linear(784, 32)
        # 隐藏层 32 -> 输出层 10 [cite: 13]
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # 展平输入 (Batch_Size, 1, 28, 28) -> (Batch_Size, 784)
        x = x.view(-1, 784)
        
        # 计算隐藏层 (这是我们要可视化的"特征频谱")
        # 使用 ReLU 激活函数 [cite: 12]
        hidden = torch.relu(self.fc1(x))
        
        # 计算输出层
        output = self.fc2(hidden)
        
        # 使用 Softmax 获取概率分布 [cite: 13]
        prob = torch.softmax(output, dim=1)
        
        # 关键点：必须同时返回 prob 和 hidden 
        # 这样 Java 端才能拿到中间层的 32 个数值画出条形码
        return prob, hidden
# ==========================================
# 2. 准备数据与训练
# ==========================================
def train():
    print("正在准备 MNIST 数据集...")
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) # 标准化
    ])
    
    # 下载训练数据
    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

    model = NanoMLP()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    print("开始训练 (演示目的只跑 1 个 Epoch)...")
    model.train()
    
    # 简单的训练循环
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        # 注意：因为 forward 返回两个值，我们这里只需要第一个值(prob)来计算 Loss
        # 但 CrossEntropyLoss 通常接收 raw logits，为了简化，这里我们做个适配
        # 或者为了更标准的训练，我们可以暂时用 output 而不是 prob 训练
        # 但为了保证导出逻辑一致，我们直接计算 loss
        
        prob, _ = model(data) 
        # CrossEntropyLoss 期望 logits，但如果用 NLLLoss 则配合 log_softmax
        # 这里为了演示简单，我们手动计算 NLL 
        loss = -torch.log(prob[range(target.shape[0]), target]).mean()
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Step: {batch_idx}/{len(train_loader)} Loss: {loss.item():.4f}')

    return model

# ==========================================
# 3. 导出 ONNX (技术闭环关键)
# ==========================================
def export_onnx(model):
    print("\n正在导出 ONNX 模型...")
    model.eval()
    
    # 1. 准备输入
    dummy_input = torch.randn(1, 1, 28, 28)
    output_path = "nano_mlp.onnx"
    
    # 2. 导出模型 (先使用 Opset 11，这是一个非常稳定且兼容性极好的版本)
    # 这一步通常不会报错，因为它不需要复杂的 14->12 转换
    torch.onnx.export(
        model, 
        (dummy_input,),
        output_path,
        verbose=False,
        input_names=['input'], 
        output_names=['prob', 'hidden'],
        opset_version=11,  # <--- 改用 11，避开 14->12 的转换 bug
        dynamic_axes={'input': {0: 'batch_size'}, 
                      'prob': {0: 'batch_size'}, 
                      'hidden': {0: 'batch_size'}}
    )
    
    # 3. 【关键修复】手动修改 IR Version
    # 之前的报错是因为你的 Python 生成了 IR v10，但 Java 只支持 IR v9
    # 我们加载模型，强制把“版本号”改写为 9，然后重新保存。
    print("正在进行兼容性降级处理 (IR Version 10 -> 9)...")
    
    onnx_model = onnx.load(output_path)
    onnx_model.ir_version = 9  # <--- 强制设置为 Java 能识别的版本
    onnx.save(onnx_model, output_path)
    
    print(f"处理完成！兼容型模型已保存为: {os.path.abspath(output_path)}")
    print("请将此新文件覆盖到 Java 项目的 src/main/resources/models/ 中")

if __name__ == '__main__':
    trained_model = train()
    export_onnx(trained_model)

正在准备 MNIST 数据集...
开始训练 (演示目的只跑 1 个 Epoch)...
Train Step: 0/938 Loss: 2.3047
Train Step: 100/938 Loss: 0.5351
Train Step: 200/938 Loss: 0.3355
Train Step: 300/938 Loss: 0.2733
Train Step: 400/938 Loss: 0.1668
Train Step: 500/938 Loss: 0.2172
Train Step: 600/938 Loss: 0.5041
Train Step: 700/938 Loss: 0.3997
Train Step: 800/938 Loss: 0.2389
Train Step: 900/938 Loss: 0.1404

正在导出 ONNX 模型...


C:\Users\XieYongyu\AppData\Local\Temp\ipykernel_36532\979594530.py:84: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0103 20:59:15.564000 36532 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 11)

正在进行兼容性降级处理 (IR Version 10 -> 9)...
处理完成！兼容型模型已保存为: d:\FILE\course\javaFX\code\nano_mlp.onnx
请将此新文件覆盖到 Java 项目的 src/main/resources/models/ 中


In [2]:
# 在 Notebook 单元格里运行
!pip freeze > requirements.txt